In [ ]:
"""Bu kod, sürücüsüz metro ağı oluşturmak ve iki istasyon arasında en hızlı ve en az aktarmalı seyahat etmek için gerekli olan süre ve rotayı bulmak için kullanılır
"""

from collections import defaultdict, deque
import heapq
from typing import Dict, List, Optional, Tuple


class Istasyon:
    def __init__(self, idx: str, ad: str, hat: str):
        self.idx = idx
        self.ad = ad
        self.hat = hat
        self.komsular = []  # Komşu istasyonlar (bağlantılar)

    def komsu_ekle(self, komsu: 'Istasyon', sure: int) -> None:
        self.komsular.append((komsu, sure))

    def __repr__(self):
        return f"{self.ad} ({self.hat})"


class MetroAgi:
    def __init__(self):
        self.istasyonlar: Dict[str, Istasyon] = {}
        self.hatlar: Dict[str, List[Istasyon]] = defaultdict(list)

    def istasyon_ekle(self, idx: str, ad: str, hat: str) -> None:
        if idx not in self.istasyonlar:
            istasyon = Istasyon(idx, ad, hat)
            self.istasyonlar[idx] = istasyon
            self.hatlar[hat].append(istasyon)

    def baglanti_ekle(self, istasyon1_id: str, istasyon2_id: str, sure: int) -> None:
        ist1 = self.istasyonlar[istasyon1_id]
        ist2 = self.istasyonlar[istasyon2_id]
        ist1.komsu_ekle(ist2, sure)
        ist2.komsu_ekle(ist1, sure)

    def en_az_aktarma_bul(self, baslangic_id: str, hedef_id: str) -> Optional[List[Istasyon]]:
        if baslangic_id not in self.istasyonlar or hedef_id not in self.istasyonlar:
            return None
        baslangic = self.istasyonlar[baslangic_id]
        hedef = self.istasyonlar[hedef_id]
        ziyaret_edildi = set()
        kuyruk = deque([(baslangic, [baslangic])])
        while kuyruk:
            istasyon, rota = kuyruk.popleft()
            if istasyon == hedef:
                return rota
            if istasyon in ziyaret_edildi:
                continue
            ziyaret_edildi.add(istasyon)
            for komsu, _ in istasyon.komsular:
                if komsu not in ziyaret_edildi:
                    kuyruk.append((komsu, rota + [komsu]))
        return None

    def en_hizli_rota_bul(self, baslangic_id: str, hedef_id: str) -> Optional[Tuple[List[Istasyon], int]]:
        if baslangic_id not in self.istasyonlar or hedef_id not in self.istasyonlar:
            return None

        baslangic = self.istasyonlar[baslangic_id]
        hedef = self.istasyonlar[hedef_id]

        def heuristic(istasyon: Istasyon, hedef: Istasyon) -> int:
            return 0

        open_set = [(0 + heuristic(baslangic, hedef), 0, id(baslangic), baslangic, [baslangic])]
        ziyaret_edildi = set()

        while open_set:
            _, toplam_sure, _, mevcut, rota = heapq.heappop(open_set)

            if mevcut == hedef:
                return rota, toplam_sure

            if id(mevcut) in ziyaret_edildi:
                continue
            ziyaret_edildi.add(id(mevcut))

            for komsu, sure in mevcut.komsular:
                if id(komsu) not in ziyaret_edildi:
                    yeni_toplam_sure = toplam_sure + sure
                    tahmini_toplam = yeni_toplam_sure + heuristic(komsu, hedef)
                    heapq.heappush(open_set, (tahmini_toplam, yeni_toplam_sure, id(komsu), komsu, rota + [komsu]))

        return None

if __name__ == "__main__":
    metro = MetroAgi()

    # İstasyonlar ekleme (Yeni duraklar: Süleymaniye, Vefa, Saraçhane, Unkapanı)
    # Mavi Hat
    metro.istasyon_ekle("S1", "Süleymaniye", "Mavi Hat")
    metro.istasyon_ekle("V1", "Vefa", "Mavi Hat")
    metro.istasyon_ekle("S2", "Saraçhane", "Mavi Hat")
    metro.istasyon_ekle("U1", "Unkapanı", "Mavi Hat")

    # Bağlantılar ekleme
    metro.baglanti_ekle("S1", "V1", 5)  # Süleymaniye -> Vefa
    metro.baglanti_ekle("V1", "S2", 4)  # Vefa -> Saraçhane
    metro.baglanti_ekle("S2", "U1", 3)  # Saraçhane -> Unkapanı

    # Test senaryoları
    print("\n=== Test Senaryoları ===")

    # Senaryo 1: Süleymaniye'den Unkapanı'na
    print("\n1. Süleymaniye'den Unkapanı'na:")
    rota = metro.en_az_aktarma_bul("S1", "U1")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

    sonuc = metro.en_hizli_rota_bul("S1", "U1")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))

    # Senaryo 2: Vefa'dan Unkapanı'na
    print("\n2. Vefa'dan Unkapanı'na:")
    rota = metro.en_az_aktarma_bul("V1", "U1")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

    sonuc = metro.en_hizli_rota_bul("V1", "U1")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))

    # Senaryo 3: Süleymaniye'den Saraçhane'ye
    print("\n3. Süleymaniye'den Saraçhane'ye:")
    rota = metro.en_az_aktarma_bul("S1", "S2")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(i.ad for i in rota))

    sonuc = metro.en_hizli_rota_bul("S1", "S2")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(i.ad for i in rota))



=== Test Senaryoları ===

1. Süleymaniye'den Unkapanı'na:
En az aktarmalı rota: Süleymaniye -> Vefa -> Saraçhane -> Unkapanı
En hızlı rota (12 dakika): Süleymaniye -> Vefa -> Saraçhane -> Unkapanı

2. Vefa'dan Unkapanı'na:
En az aktarmalı rota: Vefa -> Saraçhane -> Unkapanı
En hızlı rota (7 dakika): Vefa -> Saraçhane -> Unkapanı

3. Süleymaniye'den Saraçhane'ye:
En az aktarmalı rota: Süleymaniye -> Vefa -> Saraçhane
En hızlı rota (9 dakika): Süleymaniye -> Vefa -> Saraçhane
